In [ ]:
# for training in google colaboratory
from os import path
import sys
if 'google' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')
    # create directory
    !mkdir Data
    if not path.exists('Data/internship_data'):
        # copy gzip file from google.disk
        !cp drive/My\ Drive/Colab/NtechLab/internship_data.tar.gz Data
        # unzip file
        !tar -xf Data/internship_data.tar.gz -C Data
        # delete gzip file
        !rm Data/internship_data.tar.gz

In [ ]:
from model import AFModel
from train_functions import train, accuracy
from data_functions import train_test_split, MyDataLoader

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torchvision.datasets import ImageFolder

import albumentations as A

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# load data
dataset = ImageFolder('Data/internship_data')

In [ ]:
# split data indices to train, val and test
train_val_indices, test_indices = train_test_split(np.arange(len(dataset)),
                                                   train_size=0.75)

train_indices, val_indices = train_test_split(train_val_indices,
                                              train_size=0.75)

In [ ]:
# get classes_count
output_dim = len(dataset.classes)
# create model
afmodel = AFModel()
afmodel.create_model(None, 3, output_dim)
afmodel = afmodel.to(DEVICE)
# saving classes name in model
afmodel.save_labels(dataset.classes)

In [ ]:
# data parameters
IM_SIZE = (224, 224)
batch_size = 256
NORMALIZE = ([0.485, 0.456, 0.406],
             [0.229, 0.224, 0.225])

# Create augmentations for train data
transformer = A.Compose([
        A.Resize(*IM_SIZE),
        A.HorizontalFlip(p=0.5),
        A.OneOf([
            A.IAAAdditiveGaussianNoise(),
            A.GaussNoise(),
        ], p=0.2),
        A.OneOf([
            A.MotionBlur(p=.2),
            A.MedianBlur(blur_limit=3, p=0.1),
            A.Blur(blur_limit=3, p=0.1),
        ], p=0.2),
        A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=0.2),
        A.HueSaturationValue(p=0.3),
        A.Normalize(*NORMALIZE),
    ])

# moved axis for PyTorch
train_transformer = lambda x: np.moveaxis(transformer(image=x)['image'], -1, 0)

# Create augmentations for val and test data
transformer_test = A.Compose([A.Resize(*IM_SIZE),
                              A.Normalize(*NORMALIZE)
                            ])

val_transformer = lambda x: np.moveaxis(transformer_test(image=x)['image'], -1, 0)
# Create data loaders
train_loader = MyDataLoader(dataset, batch_size, train_indices,
                            True, train_transformer)
val_loader = MyDataLoader(dataset, batch_size, val_indices, False, val_transformer)
test_loader = MyDataLoader(dataset, batch_size, test_indices, False, val_transformer)

In [ ]:
# optimizer parameters
lr = 1e-2
WEIGHT_DECAY = 1e-5
optimizer = torch.optim.Adam(afmodel.parameters(),
                             lr=lr,
                             weight_decay=WEIGHT_DECAY)
# scheduler parameter
FACTOR = 0.3
THRESHOLD = 0.01
PATIENCE = 1

loss_func = nn.CrossEntropyLoss().to(DEVICE)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 'max', FACTOR, PATIENCE, True, THRESHOLD
)

In [ ]:
%%time
epoch_count=15
history, best_param = \
        train(afmodel, train_loader, loss_func, optimizer, epoch_count,
              accuracy, val_loader, scheduler)

In [ ]:
# Set the best model state
afmodel.load_state_dict(best_param)
# save model
afmodel.save_model()
